
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Community</a>
    </p>
</center>

# <center>Tracing with Groq</center>    

This guide demonstrates how to use Arize for monitoring and debugging your LLM using Traces and Spans. You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to send traces to Arize.

ℹ️ This notebook requires:
- A Groq API key
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [1]:
# Dependencies needed to instrument your Groq application using opentelemetry and openinference
!pip install -q "arize-otel>=0.7.0" "openinference-instrumentation-groq>=0.1.4"

# External dependencies needed to export spans and send them to our collectors: Arize
!pip install -q opentelemetry-sdk==1.28.0 opentelemetry-exporter-otlp==1.28.0 opentelemetry-instrumentation-httpx groq==0.13.0

## Step 2: Get your API Keys

Copy the Arize API_KEY and SPACE_ID from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [4]:
import os
from getpass import getpass

if not (SPACE_ID := os.getenv("SPACE_ID")):
    SPACE_ID = getpass("🔑 Enter your Space ID: ")

if not (API_KEY := os.getenv("API_KEY")):
    API_KEY = getpass("🔑 Enter your API Key: ")

if not (GROQ_API_KEY := os.getenv("GROQ_API_KEY")):
    GROQ_API_KEY = getpass("🔑 Enter your Groq API key: ")

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

## Step 3. Add our tracing code
We will be using the arize_otel package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [5]:
# Import open-telemetry dependencies
from arize.otel import register

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,  # in app space settings page
    api_key=API_KEY,  # in app space settings page
    project_name="groq-tracing-tutorial",  # name this to whatever you would like
)
# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.groq import GroqInstrumentor

# Finish automatic instrumentation
GroqInstrumentor().instrument(tracer_provider=tracer_provider)

## Step 4: Run your LLM application
Let's test our app by asking Groq to write a haiku.

In [6]:
from groq import Groq

# Run OpenAI completion
groq_client = Groq()
response = groq_client.chat.completions.create(
    model="mixtral-8x7b-32768",
    messages=[{"role": "user", "content": "Write a haiku."}],
    max_tokens=20,
)
print(response.choices[0].message.content)

Gentle spring rain falls,
Buds burst forth in vibrant green,
Nature


Great! Our application works!

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the project with the same `project_name`.

<img src="https://storage.googleapis.com/arize-assets/tutorials/images/arize-llm-tracing.png" width="1500" />